In [2]:
import pandas as pd
from sqlalchemy import event, create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
nyparking = "parking-violations-issued-fiscal-year-2016.csv"
nyparking_df = pd.read_csv(nyparking)
nyparking_df.head()

C:\Users\zhiyi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,34,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1363745270,GGY6450,99,PAS,2015-07-09T00:00:00,46,SDN,HONDA,P,0,...,WHITE,0,2010.0,-,0.0,NaN,NaN,NaN,NaN,NaN
1,1363745293,KXD355,SC,PAS,2015-07-09T00:00:00,21,SUBN,CHEVR,P,55730,...,RED,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
2,1363745438,JCK7576,PA,PAS,2015-07-09T00:00:00,21,SDN,ME/BE,P,42730,...,WHITE,0,0.0,-,0.0,NaN,NaN,NaN,NaN,NaN
3,1363745475,GYK7658,NY,OMS,2015-07-09T00:00:00,21,SUBN,NISSA,P,58130,...,BK,0,2015.0,-,0.0,NaN,NaN,NaN,NaN,NaN
4,1363745487,GMT8141,NY,PAS,2015-07-09T00:00:00,21,P-U,LINCO,P,58130,...,BLK,0,2006.0,-,0.0,NaN,NaN,NaN,NaN,NaN


In [4]:
list(nyparking_df)

['Summons Number',
 'Plate ID',
 'Registration State',
 'Plate Type',
 'Issue Date',
 'Violation Code',
 'Vehicle Body Type',
 'Vehicle Make',
 'Issuing Agency',
 'Street Code1',
 'Street Code2',
 'Street Code3',
 'Vehicle Expiration Date',
 'Violation Location',
 'Violation Precinct',
 'Issuer Precinct',
 'Issuer Code',
 'Issuer Command',
 'Issuer Squad',
 'Violation Time',
 'Time First Observed',
 'Violation County',
 'Violation In Front Of Or Opposite',
 'House Number',
 'Street Name',
 'Intersecting Street',
 'Date First Observed',
 'Law Section',
 'Sub Division',
 'Violation Legal Code',
 'Days Parking In Effect    ',
 'From Hours In Effect',
 'To Hours In Effect',
 'Vehicle Color',
 'Unregistered Vehicle?',
 'Vehicle Year',
 'Meter Number',
 'Feet From Curb',
 'Violation Post Code',
 'Violation Description',
 'No Standing or Stopping Violation',
 'Hydrant Violation',
 'Double Parking Violation']

In [5]:
nyparking_df['Date First Observed']

0                  0.0
1                  0.0
2                  0.0
3                  0.0
4                  0.0
5                  0.0
6                  0.0
7                  0.0
8                  0.0
9                  0.0
10                 0.0
11                 0.0
12                 0.0
13                 0.0
14                 0.0
15                 0.0
16                 0.0
17                 0.0
18                 0.0
19                 0.0
20                 0.0
21                 0.0
22                 0.0
23                 0.0
24                 0.0
25                 0.0
26                 0.0
27                 0.0
28                 0.0
29                 0.0
               ...    
10626869           0.0
10626870           0.0
10626871           0.0
10626872           0.0
10626873           0.0
10626874    20160607.0
10626875           0.0
10626876           0.0
10626877           0.0
10626878           0.0
10626879           0.0
10626880           0.0
10626881   

In [6]:
clean_ny_parking= nyparking_df[nyparking_df['Date First Observed'].notnull()]

In [7]:
clean_ny_parking2= clean_ny_parking[clean_ny_parking['Date First Observed'] != 0.0]

In [8]:
# Create a filtered dataframe from specific columns
parking_cols = ['Date First Observed', 'Violation Code', 'House Number','Street Name']
parking_trans= clean_ny_parking2[parking_cols].copy()
parking_trans

,Date First Observed,Violation Code,House Number,Street Name
134,20150623.0,66,NaN,STANLEY
135,20150623.0,80,NaN,STANLEY
139,20150617.0,19,74-26,BWAY
140,20150618.0,19,28-02,42 RD
173,20150712.0,24,2045,ROCKAWAY
329,20150619.0,14,430,E 105 ST
338,20150703.0,71,1694,MADISON AVE
344,20150619.0,20,211,EAST 123 ST
383,20150714.0,14,101,BARCLAY ST
532,20150609.0,46,2399,FULTON STREET


In [9]:
#Clean-up Dates
dates = []
dates = parking_trans['Date First Observed'].values
dates [1]


20150623.0

In [13]:
# get the month
MM = []
for d in dates:
    str(d)
    MM.append(d[4:5])
print(MM)

IndexError: invalid index to scalar variable.

In [ ]:
#Create Database
connection_string = "root:19060Dst;@localhost/"
engine = create_engine(f'mysql://{connection_string}')

@event.listens_for(engine, 'before_cursor_execute')
def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
    if executemany:
        cursor.fast_executemany = True
        cursor.commit()

In [ ]:
connection.execute('use etl_db;')

In [ ]:
db_connection_string = "root:19060Dst;@localhost/etl_db"
db_engine = create_engine(f'mysql://{db_connection_string}')

# Confirm tables
db_engine.table_names()

In [ ]:
parking_trans.to_sql(name='nyc_parking', con=db_engine, if_exists='replace', index=True)

In [ ]:
results = db_engine.execute('select * from nyc_parking')

In [ ]:
for item in results:
    print(item)